In [ ]:
import pandas as pd
import numpy as np
import sys
sys.path.append("..")
from dataloader_comma import CommaDataset
from expand_road_events import *
from collections import Counter
import imageio
from model import VTN
import matplotlib.pyplot as plt 
from PIL import Image
import glob
plt.rcParams.update({'font.size': 26}) 
import os
from utils import * 
import re
from vis_utils import * 
from tqdm import tqdm
import warnings 
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=UserWarning, module="matplotlib")

Checkpoint Root

In [ ]:
ckpt_root = f"/kaggle/working/ckpts_final_comma_distance_none_True_1_clip"
#find the latest version -G.R.
versions = glob.glob(os.path.join(ckpt_root, "lightning_logs", "version_*"))
if not versions:
    raise FileNotFoundError("None found")
latest_version = max(versions, key=os.path.getmtime)

# Find checkpoints in the latest version -G.R.
ckpt_files = glob.glob(os.path.join(latest_version, "checkpoints", "*.ckpt"))
if not ckpt_files:
    raise FileNotFoundError("None found.")
checkpoint_path = max(ckpt_files, key=os.path.getmtime)

print(f"Using checkpoint: {checkpoint_path}")

ckpt = torch.load(checkpoint_path)
state_dict = ckpt['state_dict']
state_dict = get_regular_ckpt_from_lightning_checkpoint(state_dict)
#added this to load the model correctly -G.R.
model.load_state_dict(state_dict)
print('done')

In [ ]:
ckpt = torch.load(checkpoint_path, map_location=gpu)

In [ ]:
state_dict = ckpt['state_dict']
state_dict = get_regular_ckpt_from_lightning_checkpoint(state_dict)
model.load_state_dict(state_dict)
model.eval()
model = model.to(gpu)

In [ ]:
def split_string(string):
    words = string.replace("a photo of driving on a highway with", "").replace("a photo of", "").replace("driving on a highway", "").replace("past", "").replace("a street with", "").split()  # Split the string into a list of words
    result = []
    current_line = ""
    
    for word in words:
        if len(current_line) + len(word) <= 33:
            current_line += word + " "
        else:
            result.append(current_line.strip())
            current_line = word + " "
    
    if current_line:
        result.append(current_line.strip())
    
    return "\n".join(result)

In [ ]:
import os

image_dir = "/kaggle/working/results_images"
gif_dir = "/kaggle/working/results_images/att"

os.makedirs(image_dir, exist_ok=True)
os.makedirs(gif_dir, exist_ok=True)


In [ ]:
sys.path.append("..")
from expand_road_events import expand_label

In [ ]:
from torch.utils.data import DataLoader
from dataloader_comma import CommaDataset

# Create the dataset
dataset_comma = CommaDataset(
    dataset_type="test",  # o "val" o "train" depends on what we want plot
    use_transform=False,
    multitask="distance",  
    ground_truth="desired", # Change to True to have all the parameters you use in the plot
    dataset_path="/kaggle/input/final-hdf5-files",
    dataset_fraction=1.0
)

print(f"Dataset created with {len(dataset_comma)} samples")

# Creating the Dataloader
dataloader_comma = DataLoader(
    dataset_comma,
    batch_size=1,  # Keep 1 for plotting
    shuffle=False,  # Don't shuffle for plots
    num_workers=0,  # 0 for easier debugging
    # collate_fn=None  # Use the default collate_fn, NOT the custom one you showed
)

print("DataLoader created successfully")


In [ ]:
from utils import get_scenarios

scenarios, scenarios_tokens = get_scenarios("retinanet")   #or clip

scenarios = list(scenarios)

In [ ]:
for j, batch in enumerate(dataloader_comma):
    image_array,  vego, angle, distance, g, s, l, seq_key = batch #g= gaspressed, s= brakepressed, l= cruiseenabled
        
    img = image_array
    max_len = 240
    # subito dopo il batch unpacking
    img, angle, distance, vego = img.to(gpu), angle.to(gpu), distance.to(gpu), vego.to(gpu)
    (logits, attns), concepts = model(img, angle, distance, vego, seq_key)
    
    top5_indices = torch.tensor(concepts.squeeze()).topk(10).indices
    s = img.shape
    angle, distance, vego, logits, concepts = angle.to("cpu"), distance.to("cpu"), vego.to("cpu"), logits.detach().cpu().to("cpu"), concepts.detach().cpu().to("cpu")
    
    f = []
    inter = []
    for i, elem0 in enumerate(top5_indices):
        inter = []
        for elem in top5_indices[max(i-20, 0):min(i+20,len(top5_indices))]:
            l = elem.cpu().numpy().tolist()
            if 131 in l:
                l.remove(131)
            if 55 in l:
                l.remove(55)
            inter.extend(l)
        count_dict = Counter(inter)
        # Get the top 5 most occurring numbers
        top_5 = count_dict.most_common(3)
        intermediate = []
        for a in top_5: 
            intermediate.append(scenarios[a[0]])
        f.append(intermediate)

    fig, axes = plt.subplots(1, 1, figsize=(12, 16))#,gridspec_kw= {'height_ratios': [20, 1]})

    plt_idx = 0
    for i, image in tqdm(enumerate(img[0][10:120])):
    
        image_frame = (image).cpu().permute(1, 2, 0)#unorm(image).cpu().permute(1, 2, 0)

        # Display the image frame
        axes.imshow((np.array(image_frame) * 255).astype(np.uint8))
        
        #title = '\n'.join([split_string("\u2022 " + h) for h in f[i]]) # ORIGINAL
        lines = [split_string(h) for h in f[i]]               # senza bullet
        expanded = [expand_label(line) for line in lines]     # espandi le frasi
        title = '\n'.join("\u2022 " + e for e in expanded)    # riaggiungi il bullet
                
        axes.set_title(title)
        axes.set_aspect('equal')
        axes.set_xticks([])
        axes.set_yticks([])

        # Remove borders
        axes.spines['top'].set_visible(False)
        axes.spines['bottom'].set_visible(False)
        axes.spines['left'].set_visible(False)
        axes.spines['right'].set_visible(False)

        plt.savefig(f"/kaggle/working/results_images/{i}.png") #Modified to save images in Kaggle environment

    
    image_directory = '/kaggle/working/results_images/v2'

    # Set the output GIF file path
    output_path = lambda x: f'/kaggle/working/results_images/att/attention_comma_{j}.{x}'

    # Set the duration (in milliseconds) for each frame in the GIF
    frame_duration = 700

    # Get a sorted list of image files in the directory
    image_files = sorted(glob.glob(f'{image_directory}/*.png'), key=extract_number)  # Adjust the file extension if necessary
    
    # Create a list to store the frames of the GIF
    frames = []

    # Iterate over each image file
    for image_file in image_files:
        # Open the image file
        image = Image.open(image_file)

        # Add the image to the list of frames
        frames.append(image)

    # Save the frames as a GIF
    #frames[0].save(output_path("gif"), format='GIF', append_images=frames[1:], save_all=True,
    #            duration=frame_duration, loop=0)
    #imageio.mimsave(output_path("mp4"), frames, fps=4)
    
    if j > 5: break

In [ ]:
# ============================
# Secure plotting + marker e zoom
# ============================
import os, gc, glob
import numpy as np
import torch
import matplotlib
matplotlib.use('Agg')   
import matplotlib.pyplot as plt
from matplotlib import patches
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from scipy.signal import find_peaks
from collections import Counter
from PIL import Image
import imageio
from tqdm import tqdm

# ---------------- Parameters (modify if needed) ----------------
outdir = "/kaggle/working/results_images_safe"
os.makedirs(outdir, exist_ok=True)

chunk_size = 8            # number of columns per figure
fig_w_per_col = 4.0
fig_h = 6.0
dpi_save = 200
make_gif = True
normalize_speed = False   # if True normalizes each speed_graph (useful if curves are very flat)
save_model_checkpoint = False
checkpoint_path = "/kaggle/working/model_ckpt.pth"
verbose = True

# New parameters for marker/zoom
zoom_window = 20          # +/- steps shown around the current frame (adjust here)
marker_size = 48
marker_color = "red"
annotate_values = True    # shows numerical value next to the marker
# --------------------------------------------------------------

# helper safe save (draw before save to avoid NoneType renderer)
def safe_plot_and_save(fig, png_path, dpi=dpi_save):
    try:
        fig.canvas.draw()               # builds renderer
        fig.savefig(png_path, dpi=dpi, bbox_inches='tight')
    except Exception as e:
        try:
            # fallback: tries without tight bbox/inset
            print(f"[WARN] primary save failed: {e}. Retrying fallback.")
            fig.savefig(png_path, dpi=dpi)
        except Exception as e2:
            print(f"[ERROR] save fallback failed: {e2}")
            raise e2
    finally:
        plt.close(fig)

# simple split string for titles
def split_string(s, max_len=40):
    return "\n".join([s[i:i+max_len] for i in range(0, len(s), max_len)])

# model device
device = next(model.parameters()).device
print(f"[INFO] model device: {device}")

saved_pngs = []

# ---------- loop for batch (here we process 1 batch for debugging; remove break for all) ----------
for j, batch in enumerate(dataloader_comma):
    # your dataloader returns 7 elements: images, vego, angle, distance, gas, brake, cruise
    image_array, vego, angle, distance, gas, brake, cruise, seq_key = batch

    # move input to model device and do forward
    img = image_array.to(device)
    angle_dev = angle.to(device); distance_dev = distance.to(device); vego_dev = vego.to(device)

    # forward (try/except if model can fail)
    with torch.no_grad():
        (logits, attns), concepts = model(img, angle_dev, distance_dev, vego_dev, seq_key)

    # move to CPU for plotting (img_cpu shape: [B, seq_len, C, H, W])
    img_cpu = img.detach().cpu()

    # concepts on CPU
    concepts_cpu = concepts.detach().cpu()

    # attns on CPU (gestisce list/tuple o None)
    if attns is None:
        attns_cpu = []
    elif isinstance(attns, (list, tuple)):
        attns_cpu = [a.detach().cpu() for a in attns]
    else:
        attns_cpu = [attns.detach().cpu()]

    # function that produces speed graph from an attention layer (catch errors)
    def compute_speed_from_att(atten_cpu):
        try:
            seq_len_local = atten_cpu.shape[2]
            alignment_array = get_aligned_attention(atten_cpu.squeeze().cpu(), seq_len_local)
            g = alignment_array.sum(axis=0)[8:-8]
            return moving_average(g, 10)
        except Exception as e:
            if verbose: print(f"[WARN] compute_speed failed: {e}")
            return np.array([])

    # compute speed_graphs for the first 3 available elements (otherwise empty array)
    speed_graphs = []
    for idx in range(3):
        if idx < len(attns_cpu):
            sg = compute_speed_from_att(attns_cpu[idx][:,:,0:concepts_cpu.shape[1]])
        else:
            sg = np.array([])
        speed_graphs.append(np.array(sg))
    sg0, sg1, sg2 = speed_graphs

    # diagnostic
    if verbose:
        for k, sg in enumerate((sg0, sg1, sg2)):
            if sg.size:
                print(f"[BATCH {j}] sg{k} min/max/sum: {sg.min():.6f}, {sg.max():.6f}, {sg.sum():.6f}")
            else:
                print(f"[BATCH {j}] sg{k} empty")

    #nes = [split_string(h) for h in f[i]] # senza bullet
    #expanded = [expand_label(line) for line in lines] # espandi le frasi 
    #title = '\n'.join("\u2022 " + e for e in expanded) # riaggiungi il bullet

    # title composition (top concepts) 
    topk = torch.tensor(concepts_cpu.squeeze()).topk(5).indices
    f = []
    for i, _ in enumerate(topk):
        inter = []
        for elem in topk[max(i-5, 0):min(i+5, len(topk))]:
            l = elem.cpu().numpy().tolist()
            for bad in (131, 55):
                if bad in l:
                    l.remove(bad)
            inter.extend(l)
        cdict = Counter(inter)
        top_2 = cdict.most_common(2)
        f.append([expand_label(scenarios[a[0]]) for a in top_2])

    # optional: build intervention mask for highlighting (use gas/brake/cruise)
    try:
        intervention_mask = (np.array(gas).astype(bool) | np.array(brake).astype(bool) | (~np.array(cruise).astype(bool)))
    except Exception:
        intervention_mask = None

    # indices of the frames to plot
    seq_len = img_cpu.shape[1]
    frames_indices = list(range(10, seq_len - 10))
    if len(frames_indices) == 0:
        print("[WARN] nessun frame utile (range 10:-10 vuoto). Skip batch.")
        # pulizia e continua
        del img, angle_dev, distance_dev, vego_dev, logits, concepts, attns
        gc.collect(); torch.cuda.empty_cache()
        continue


    def maybe_normalize(arr):
        if arr.size == 0: return arr
        if not normalize_speed: return arr
        mn, mx = arr.min(), arr.max()
        if mx - mn < 1e-9: return np.zeros_like(arr)
        return (arr - mn) / (mx - mn + 1e-12)

    sg0n = maybe_normalize(sg0); sg1n = maybe_normalize(sg1); sg2n = maybe_normalize(sg2)
    lengths = [len(x) for x in (sg0n, sg1n, sg2n) if len(x) > 0]
    L = max(lengths) if lengths else (len(sg0n) if len(sg0n)>0 else 1)
    x_global = np.arange(L)

    # -------- plotting chunked & safe saving --------
    for chunk_start in range(0, len(frames_indices), chunk_size):
        chunk = frames_indices[chunk_start: chunk_start + chunk_size]
        ncol = len(chunk)
        if ncol == 0:
            continue

        fig, axes = plt.subplots(2, ncol, figsize=(fig_w_per_col * ncol, fig_h),
                                 gridspec_kw={'height_ratios':[3,1]})
        axes = np.array(axes)
        axes_flat = axes.flatten()


        sgs_for_combined = [np.array(a) for a in (sg0n, sg1n, sg2n) if a.size>0]
        all_arr = np.concatenate(sgs_for_combined) if len(sgs_for_combined)>0 else np.array([0.0,1.0])
        global_min, global_max = float(all_arr.min()), float(all_arr.max())
        if abs(global_max - global_min) < 1e-9:
            global_min -= 0.5; global_max += 0.5

        # precompute combined (interpolato su x_global)
        combined = np.zeros(len(x_global))
        for a in (sg0n, sg1n, sg2n):
            if a.size > 0:
                combined[:len(a)] += np.interp(x_global, np.linspace(0, len(a)-1, len(a)), a)
        peaks, _ = find_peaks(combined, distance=max(4, len(x_global)//20)) if len(x_global)>1 else ([], {})

        for k, frame_idx in enumerate(chunk):
            ax_img = axes_flat[k]
            ax_plot = axes_flat[k + ncol]

            img_tensor = img_cpu[0][frame_idx]
            image_frame = img_tensor.permute(1,2,0).numpy()

            # image pane
            ax_img.imshow((image_frame * 255).astype(np.uint8))
            title = '\n'.join([split_string("• " + h) for h in (f[frame_idx - 10] if 0 <= frame_idx-10 < len(f) else [])])
            ax_img.set_title(title, fontsize=9)
            ax_img.axis("off")

            # plot heads (interpolando su x_global)
            plotted_any = False
            heads = [(sg0n, "#1f77b4"), (sg1n, "#ff7f0e"), (sg2n, "#2ca02c")]
            linestyles = ["-", "--", "-."]
            linewidths = [2.0, 1.8, 1.8]


            head_plots = []
            for idx_head, (sg_arr, col) in enumerate(heads):
                if sg_arr.size == 0:
                    head_plots.append(None)
                    continue
                sg_plot = np.interp(x_global, np.linspace(0, len(sg_arr)-1, len(sg_arr)), sg_arr) if len(sg_arr)!=len(x_global) else sg_arr
                ax_plot.plot(x_global, sg_plot, label=f'Head {idx_head+1}', color=col, linestyle=linestyles[idx_head], linewidth=linewidths[idx_head], alpha=0.95)
                head_plots.append(sg_plot)
                plotted_any = True

            # mark peaks (global)
            for p in peaks:
                if 0 <= p < len(x_global):
                    ax_plot.plot(p, combined[p], marker='o', markersize=4, color='darkred', alpha=0.9)

            pos = frame_idx - 10
            
            pos_clipped = int(np.clip(pos, 0, len(x_global)-1))
            if 0 <= pos_clipped < len(x_global):
         
                for idx_head, sg_plot in enumerate(head_plots):
                    if sg_plot is None: 
                        continue
                    y_val = float(np.interp(pos_clipped, x_global, sg_plot))
                    ax_plot.scatter(pos_clipped, y_val, color=marker_color, s=marker_size, zorder=5)
                    if annotate_values:
                        ax_plot.annotate(f"{y_val:.2f}", xy=(pos_clipped, y_val), xytext=(3, 3),
                                         textcoords="offset points", fontsize=7, color=marker_color)

                x_min = max(0, pos_clipped - zoom_window)
                x_max = min(len(x_global)-1, pos_clipped + zoom_window)
               
                y_vals_chunk = []
                for sg_plot in head_plots:
                    if sg_plot is not None:
                        start = x_min; end = x_max+1
                        y_vals_chunk.extend(sg_plot[start:end].tolist())
                if len(y_vals_chunk) > 0:
                    y_min_z = min(y_vals_chunk); y_max_z = max(y_vals_chunk)
                    ypad = max(1e-6, (y_max_z - y_min_z) * 0.15)
                    ax_plot.set_ylim(y_min_z - ypad, y_max_z + ypad)
                ax_plot.set_xlim(x_min, x_max)

           
            if (intervention_mask is not None) and (len(intervention_mask) == img_cpu.shape[1]):
                seg_on=False; start=None
                for t_i, val in enumerate(intervention_mask):
                    if val and not seg_on: seg_on=True; start=t_i
                    if (not val) and seg_on: seg_on=False; end=t_i; 
                    if 'end' in locals():
                        ax_plot.add_patch(patches.Rectangle((start, ax_plot.get_ylim()[0]), end-start, ax_plot.get_ylim()[1]-ax_plot.get_ylim()[0], color='magenta', alpha=0.06, linewidth=0))
                if seg_on:
                    end = len(intervention_mask)-1
                    ax_plot.add_patch(patches.Rectangle((start, ax_plot.get_ylim()[0]), end-start, ax_plot.get_ylim()[1]-ax_plot.get_ylim()[0], color='magenta', alpha=0.06, linewidth=0))

           
            if plotted_any:
                ax_plot.legend(fontsize=8, loc='upper right')
            else:
                ax_plot.text(0.5, 0.5, "No attention data", ha='center', va='center', fontsize=9, color='gray')
                ax_plot.set_ylim(0, 1)

            ax_plot.set_xlabel("Sequence position", fontsize=8)
            ax_plot.set_ylabel("Attention activation", fontsize=8)
            ax_plot.tick_params(axis='both', which='major', labelsize=7)
            ax_plot.grid(axis='y', linestyle=':', linewidth=0.5, alpha=0.6)

        plt.tight_layout()
        png_path = os.path.join(outdir, f"batch{j}_chunk{chunk_start}.png")
        safe_plot_and_save(fig, png_path, dpi=dpi_save)
        saved_pngs.append(png_path)

 
    if make_gif and saved_pngs:
        try:
            frames = [Image.open(p) for p in saved_pngs]
            gif_path = os.path.join(outdir, f"attention_batch{j}.gif")
            frames[0].save(gif_path, format='GIF', append_images=frames[1:], save_all=True, duration=500, loop=0)
            for im in frames: im.close()
            print(f"[INFO] GIF creata: {gif_path}")
        except Exception as e:
            print(f"[WARN] creazione GIF fallita: {e}")

   
    del img, angle_dev, distance_dev, vego_dev, logits, concepts, attns
    gc.collect()
    torch.cuda.empty_cache()

  
    if save_model_checkpoint:
        try:
            torch.save(model.state_dict(), checkpoint_path)
            print(f"[INFO] model saved in {checkpoint_path}")
        except Exception as e:
            print(f"[WARN] save model failed: {e}")
        
        try:
            del model
            gc.collect()
            torch.cuda.empty_cache()
            print("[INFO] model deleted from memory.")
        except Exception:
            pass

  
    print("[DONE] saved pngs:", saved_pngs)
    if saved_pngs:
        from IPython.display import display
        display(Image.open(saved_pngs[0]))

    break   
